In [80]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from class_function_V1 import TimeSeriesAnalysis #Função que processa os dados, separando no grupo de 53 setores
import numpy as np
from pySSA.core import MSSA
from pySSA.simple import SSA


# Leitura do arquivo Excel
erro_prev_SRM = pd.read_excel(r"Tabelas\Forecasting_SARIMA.xlsx")
erro_prev_HW = pd.read_excel(r"Tabelas\Forecasting_Holt_winters.xlsx")
erro_prev_HW.drop('RRMSE_Out-of-Sample', axis=1) 
param_HW = pd.read_excel(r"Tabelas\Param_Holt_winters.xlsx")
param_SRM = pd.read_excel(r"Tabelas\Param_SARIMA.xlsx")
#erro_prev_SRM.head()
param_HW.head()

,Setor,Best Metric,trend,Seasonal,Seasonal_Periods,Modelo,MSE_In-Sample,EQA_In-Sample,RMSE_In-Sample,RRMSE_In-Sample,MAE_In-Sample,MAPE_In-Sample,MSE_Out-of-Sample,EQA_Out-of-Sample,RMSE_Out-of-Sample,RRMSE_Out-of-Sample,MAE_Out-of-Sample,MAPE_Out-of-Sample
0,10.1,0.001924,add,mul,12,<statsmodels.tsa.holtwinters.results.HoltWinte...,0.001631,0.370313,0.040390,0.008758,0.030935,0.006722,0.001924,0.048098,0.043863,0.009448,0.033816,0.007288
1,10.3,0.026120,add,add,12,<statsmodels.tsa.holtwinters.results.HoltWinte...,0.039954,9.069664,0.199886,0.046170,0.145174,0.036646,0.026120,0.653007,0.161618,0.039902,0.122460,0.032932
2,10.4,0.002711,mul,mul,12,<statsmodels.tsa.holtwinters.results.HoltWinte...,0.003979,0.903281,0.063081,0.013676,0.047272,0.010367,0.002711,0.067776,0.052068,0.011089,0.042074,0.009059
3,10.5,0.017370,mul,add,36,<statsmodels.tsa.holtwinters.results.HoltWinte...,0.001438,0.326373,0.037918,0.008300,0.029893,0.006557,0.017370,0.434256,0.131796,0.030239,0.125681,0.028883
4,10.6,0.001198,add,mul,24,<statsmodels.tsa.holtwinters.results.HoltWinte...,0.000977,0.221891,0.031265,0.006833,0.024264,0.005314,0.001198,0.029952,0.034613,0.007525,0.029606,0.006437


In [76]:
import pandas as pd

def compare_models_predict(errors_hw_df, errors_srm_df, metric):
    # Count the cases where each model has the lowest error for each h value
    count_df = pd.DataFrame(columns=['h (Forecast Period)'])

    h_values = [1, 6, 9, 12]

    for h in h_values:
        hw_better_count_in_sample = ((errors_hw_df['h(período da previsão)'] == h) & (errors_hw_df[f'{metric}_In-Sample'] < errors_srm_df[f'{metric}_In-Sample'])).sum()
        srm_better_count_in_sample = ((errors_srm_df['h(período da previsão)'] == h) & (errors_srm_df[f'{metric}_In-Sample'] < errors_hw_df[f'{metric}_In-Sample'])).sum()

        hw_better_count_out_of_sample = ((errors_hw_df['h(período da previsão)'] == h) & (errors_hw_df[f'{metric}_Out-of-Sample'] < errors_srm_df[f'{metric}_Out-of-Sample'])).sum()
        srm_better_count_out_of_sample = ((errors_srm_df['h(período da previsão)'] == h) & (errors_srm_df[f'{metric}_Out-of-Sample'] < errors_hw_df[f'{metric}_Out-of-Sample'])).sum()

        count_df = count_df.append({'h (Forecast Period)': h, 'HW Better Count (In-Sample)': hw_better_count_in_sample, 'SRM Better Count (In-Sample)': srm_better_count_in_sample, 'HW Better Count (Out-of-Sample)': hw_better_count_out_of_sample, 'SRM Better Count (Out-of-Sample)': srm_better_count_out_of_sample}, ignore_index=True)

    # Display the counts for each h value
    print(f"Counts for Metric: {metric}")
    count_df.head()
    print(count_df)
    # print()

    # Find the best model for each sector
    sectors = errors_hw_df['Setor'].unique()
    best_model_df = pd.DataFrame(columns=['Setor', 'Best Model'])

    for sector in sectors:
        hw_better_count_in_sample = ((errors_hw_df['Setor'] == sector) & (errors_hw_df[f'{metric}_In-Sample'] < errors_srm_df[f'{metric}_In-Sample'])).sum()
        srm_better_count_in_sample = ((errors_srm_df['Setor'] == sector) & (errors_srm_df[f'{metric}_In-Sample'] < errors_hw_df[f'{metric}_In-Sample'])).sum()

        hw_better_count_out_of_sample = ((errors_hw_df['Setor'] == sector) & (errors_hw_df[f'{metric}_Out-of-Sample'] < errors_srm_df[f'{metric}_Out-of-Sample'])).sum()
        srm_better_count_out_of_sample = ((errors_srm_df['Setor'] == sector) & (errors_srm_df[f'{metric}_Out-of-Sample'] < errors_hw_df[f'{metric}_Out-of-Sample'])).sum()

        if hw_better_count_in_sample + hw_better_count_out_of_sample > srm_better_count_in_sample + srm_better_count_out_of_sample:
            best_model = 'HW'
        elif hw_better_count_in_sample + hw_better_count_out_of_sample < srm_better_count_in_sample + srm_better_count_out_of_sample:
            best_model = 'SRM'
        else:
            best_model = 'Tie'

        best_model_df = best_model_df.append({'Setor': sector, 'Best Model': best_model,'HW Better Count (In-Sample)': hw_better_count_in_sample, 'SRM Better Count (In-Sample)': srm_better_count_in_sample, 'HW Better Count (Out-of-Sample)': hw_better_count_out_of_sample, 'SRM Better Count (Out-of-Sample)': srm_better_count_out_of_sample}, ignore_index=True)

    # # Display the best model for each sector
    # print(f"Best Model for Metric: {metric}")
    # print(best_model_df)


In [90]:
compare_models_predict(erro_prev_HW,erro_prev_SRM,'MSE')

Counts for Metric: MSE
  h (Forecast Period)  HW Better Count (In-Sample)  \
0                   1                         45.0   
1                   6                         45.0   
2                   9                         45.0   
3                  12                         45.0   

   HW Better Count (Out-of-Sample)  SRM Better Count (In-Sample)  \
0                             31.0                           7.0   
1                             25.0                           7.0   
2                             25.0                           7.0   
3                             22.0                           7.0   

   SRM Better Count (Out-of-Sample)  
0                              21.0  
1                              27.0  
2                              27.0  
3                              30.0  


---------------

In [84]:
import pandas as pd

def compare_models_param(errors_hw_df, errors_srm_df, metric):
    hw_better_count = 0
    srm_better_count = 0

    for index, row in errors_hw_df.iterrows():
        hw_error = row[f'{metric}_Out-of-Sample']
        srm_error = errors_srm_df.loc[index, f'{metric}_Out-of-Sample']
        
        if hw_error < srm_error:
            hw_better_count += 1
        elif hw_error > srm_error:
            srm_better_count += 1

    print(f"Counts for Metric: {metric}")
    print(f"HW Better Count: {hw_better_count}")
    print(f"SRM Better Count: {srm_better_count}")


In [89]:
compare_models_param(param_HW,param_SRM,'RMSE')

Counts for Metric: RMSE
HW Better Count: 15
SRM Better Count: 37
